# Step 1: Imports

In [1]:
from typing import List, Tuple
import numpy as np
import pandas as pd

pd.set_option("display.max_columns", None)

def load_raw_data(filename: str) -> pd.DataFrame:
    mens_filepath = f"/kaggle/input/warmup-round-march-machine-learning-mania-2023/M{filename}.csv"
    weomens_filepath = f"/kaggle/input/warmup-round-march-machine-learning-mania-2023/W{filename}.csv"
    df_mens = pd.read_csv(mens_filepath)
    df_mens["Gender"] = 0
    df_weomens = pd.read_csv(weomens_filepath)
    df_weomens["Gender"] = 1
    return pd.concat([df_mens, df_weomens])

def process_detailed_results(detailed_results: pd.DataFrame) -> pd.DataFrame:
    df = detailed_results.copy()
    df = clean_detailed_results(df)
    df = game_to_team_conversion(df)
    df = enrich_team_results(df)
    df = transform_team_results(df)
    return df

def clean_detailed_results(df: pd.DataFrame) -> pd.DataFrame:
    return df.drop(["WLoc", "DayNum", "Gender"], axis=1)

def game_to_team_conversion(game_results: pd.DataFrame) -> pd.DataFrame:
    winners = rename_columns(game_results, "W")
    loosers = rename_columns(game_results, "L")
    team_results = pd.concat((winners, loosers))
    team_results.drop(["TeamIDOpp"], axis=1, inplace=True)
    return team_results

def enrich_team_results(df: pd.DataFrame) -> pd.DataFrame:
    df["Dur"] = 40 + 5 * df["NumOT"]
    df.drop(["NumOT"], axis=1, inplace=True)
    return df

def transform_team_results(df: pd.DataFrame) -> pd.DataFrame:
    df = df.groupby(["Season", "TeamID"]).sum()
    df["ScorePM"] = df["Score"] / df["Dur"]
    df["ScorePMOpp"] = df["ScoreOpp"] / df["Dur"]
    df["FGAPM"] = df["FGA"] / df["Dur"]
    df["FGAPMOpp"] = df["FGAOpp"] / df["Dur"]
    df["FGA3PM"] = df["FGA3"] / df["Dur"]
    df["FGA3PMOpp"] = df["FGA3Opp"] / df["Dur"]
    df["FTAPM"] = df["FTA"] / df["Dur"]
    df["FTAPMOpp"] = df["FTAOpp"] / df["Dur"]
    df["ORPM"] = df["OR"] / df["Dur"]
    df["ORPMOpp"] = df["OROpp"] / df["Dur"]
    df["DRPM"] = df["DR"] / df["Dur"]
    df["DRPMOpp"] = df["DROpp"] / df["Dur"]
    df["AstPM"] = df["Ast"] / df["Dur"]
    df["AstPMOpp"] = df["AstOpp"] / df["Dur"]
    df["TOPM"] = df["TO"] / df["Dur"]
    df["TOPMOpp"] = df["TOOpp"] / df["Dur"]
    df["StlPM"] = df["Stl"] / df["Dur"]
    df["StlPMOpp"] = df["StlOpp"] / df["Dur"]
    df["BlkPM"] = df["Blk"] / df["Dur"]
    df["BlkPMOpp"] = df["BlkOpp"] / df["Dur"]
    df["PFPM"] = df["PF"] / df["Dur"]
    df["PFPMOpp"] = df["PFOpp"] / df["Dur"]
    df["FGSR"] = df["FGM"] / df["FGA"]
    df["FGSROpp"] = df["FGMOpp"] / df["FGAOpp"]
    df["FG3SR"] = df["FGM3"] / df["FGA3"]
    df["FG3SROpp"] = df["FGM3Opp"] / df["FGA3Opp"]
    df["FTSR"] = df["FTM"] / df["FTA"]
    df["FTSROpp"] = df["FTMOpp"] / df["FTAOpp"]
    drop_columns = ["Score", "FGM", "FGA", "FGM3", "FGA3", "FTM", "FTA", "OR", "DR", "Ast", "TO", "Stl", "Blk", "PF"]
    drop_columns += [f"{name}Opp" for name in drop_columns]
    drop_columns.append("Dur")
    df.drop(drop_columns, axis=1, inplace=True)
    return df.reset_index()
    
def rename_columns(df: pd.DataFrame, team_prefix: str) -> pd.DataFrame:
    df = df.copy()
    df.columns =  (rename_column(column_name, team_prefix) for column_name in df.columns)
    return df

def rename_column(column_name: str, team_prefix: str) -> pd.DataFrame:
    if team_prefix == "W":
        opponent_prefix = "L"
    elif team_prefix == "L":
        opponent_prefix = "W"
    else:
        raise ValueError
    if column_name.startswith(team_prefix):
        column_name = column_name.lstrip(team_prefix)
    elif column_name.startswith(opponent_prefix):
        column_name = f"{column_name.lstrip(opponent_prefix)}Opp"
    return column_name

def split_winner_and_looser_columns(df: pd.DataFrame) -> Tuple[List[str], List[str]]:
    winner_columns = [name for name in df.columns if not name.startswith("L")]
    looser_columns = [name for name in df.columns if not name.startswith("W")]
    return winner_columns, looser_columns

def clean_column_names(df: pd.DataFrame) -> List[str]:
    column_names = [
        name[1:] if 
        name.startswith("L") or name.startswith("W")
        else name 
        for name in df.columns
    ]
    return column_names

def process_seeds(df_in: pd.DataFrame) -> pd.DataFrame:
    df = df_in.copy()
    mask = df["Season"] > 2002
    df = df[mask]
    df["Seed"] = df["Seed"].str.replace(r"\D+","", regex=True)
    df["Seed"] = df["Seed"].astype(int)
    return df

def merge_features(season_features: pd.DataFrame, tournament_features: pd.DataFrame, seed_features: pd.DataFrame) -> pd.DataFrame:
    features = pd.merge(
        season_features,
        tournament_features,
        how="inner",
        on=["Season", "TeamID"],
        suffixes=("Reg", "Tou")
    )
    features = features.merge(
        seed_features,
        how="inner",
        on=["Season", "TeamID"]
    )
    return features

def get_outcomes(df):
    input_rows = df.to_records()
    output_rows = []
    for input_row in input_rows:
        output_rows.extend(parse_row(input_row))
    out_df = pd.DataFrame(output_rows)
    out_df = out_df[out_df["Season"] < 2017]
    return out_df

def parse_row(row):
    season = row['Season']
    winning_team_id = row['WTeamID']
    losing_team_id = row['LTeamID']
    if winning_team_id < losing_team_id:
        small_id = winning_team_id
        big_id = losing_team_id
        outcome = True
    elif losing_team_id < winning_team_id:
        small_id = losing_team_id
        big_id = winning_team_id
        outcome = False
    records = [
        {
            "ID": f"{season}_{small_id}_{big_id}",
            'Season': season,
            'LowID': small_id,
            'HighID': big_id,
            'Win': outcome
        },
        {
            "ID": f"{season}_{big_id}_{small_id}",
            'Season': season,
            'LowID': big_id,
            'HighID': small_id,
            'Win': not outcome
        },
    ]
    return records

def merge_outcomes_with_features(outcomes: pd.DataFrame, features: pd.DataFrame, how: str = "inner") -> pd.DataFrame:
    data = pd.merge(
        outcomes, 
        features, 
        how=how, 
        left_on=["Season", "HighID"], 
        right_on=["Season", "TeamID"]
    )
    data = pd.merge(
        data, 
        features, 
        how=how, 
        left_on=["Season", "LowID"], 
        right_on=["Season", "TeamID"],
        suffixes=("High", "Low")
    )
    data.drop(
        ["Season", "HighID", "LowID","TeamIDHigh","TeamIDLow"], 
        axis=1, 
        inplace=True
    )
    data.set_index("ID", inplace=True)
    return data

# Step 2: Load the data

## Season Detailed Results

In [2]:
RegularSeasonDetailedResults = load_raw_data("RegularSeasonDetailedResults")
RegularSeasonDetailedResults.tail()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF,Gender
70673,2023,127,3415,63,3142,54,N,0,20,56,2,10,21,30,11,23,9,14,5,2,22,17,43,4,13,16,21,1,21,5,15,7,2,24,1
70674,2023,127,3424,71,3361,68,H,0,23,55,2,12,23,33,10,23,12,14,7,6,16,28,58,5,10,7,10,7,24,12,13,4,1,26,1
70675,2023,127,3455,65,3378,53,A,0,24,51,6,13,11,13,2,28,14,11,7,2,12,19,60,4,22,11,13,8,22,11,12,2,6,12,1
70676,2023,127,3461,65,3161,56,H,0,25,57,5,17,10,16,13,35,15,14,3,2,13,21,55,9,24,5,8,1,20,12,8,8,4,17,1
70677,2023,127,3477,65,3230,62,A,0,23,50,3,13,16,19,12,20,10,9,6,0,11,22,51,8,22,10,12,8,15,12,9,5,1,13,1


In [3]:
season_features = process_detailed_results(RegularSeasonDetailedResults)
season_features.tail()

,Season,TeamID,ScorePM,ScorePMOpp,FGAPM,FGAPMOpp,FGA3PM,FGA3PMOpp,FTAPM,FTAPMOpp,ORPM,ORPMOpp,DRPM,DRPMOpp,AstPM,AstPMOpp,TOPM,TOPMOpp,StlPM,StlPMOpp,BlkPM,BlkPMOpp,PFPM,PFPMOpp,FGSR,FGSROpp,FG3SR,FG3SROpp,FTSR,FTSROpp
12130,2023,3473,1.366000,1.801000,1.258000,1.492000,0.525000,0.421000,0.329000,0.400000,0.159000,0.266000,0.481000,0.551000,0.297000,0.353000,0.482000,0.343000,0.158000,0.270000,0.048000,0.073000,0.401000,0.410000,0.392687,0.463807,0.304762,0.287411,0.662614,0.740000
12131,2023,3474,1.427885,1.793269,1.451923,1.374038,0.417308,0.383654,0.463462,0.585577,0.211538,0.235577,0.513462,0.675000,0.163462,0.352885,0.378846,0.400962,0.180769,0.164423,0.058654,0.083654,0.529808,0.444231,0.345033,0.456963,0.264977,0.325815,0.680498,0.704433
12132,2023,3475,1.552153,1.638278,1.374163,1.395215,0.386603,0.517703,0.465072,0.531100,0.223923,0.230622,0.591388,0.538756,0.323445,0.306220,0.495694,0.430622,0.187560,0.277512,0.069856,0.079426,0.461244,0.487081,0.394847,0.396433,0.284653,0.310536,0.767490,0.699099
12133,2023,3476,1.472889,1.628444,1.389333,1.454222,0.504000,0.393778,0.338667,0.360000,0.216000,0.218667,0.561778,0.571556,0.348444,0.293333,0.361778,0.279111,0.096889,0.196444,0.067556,0.080000,0.372444,0.401778,0.386436,0.430929,0.305115,0.316027,0.724409,0.696296
12134,2023,3477,1.620968,1.748387,1.466935,1.533871,0.361290,0.495968,0.556452,0.366129,0.287097,0.237097,0.592742,0.586290,0.295968,0.389516,0.415323,0.396774,0.201613,0.222581,0.080645,0.104032,0.389516,0.466935,0.393073,0.432702,0.254464,0.331707,0.675362,0.700441


## Tournament Detailed Results

In [4]:
NCAATourneyDetailedResults = load_raw_data("NCAATourneyDetailedResults")
NCAATourneyDetailedResults.tail()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF,Gender
755,2022,147,3163,91,3301,87,N,2,37,77,5,21,12,20,12,23,10,7,5,2,16,32,66,7,23,16,19,6,30,20,13,4,7,16,1
756,2022,147,3257,62,3276,50,N,0,25,58,5,15,7,9,6,20,12,11,15,4,17,16,46,3,14,15,20,10,24,9,21,6,2,12,1
757,2022,151,3163,63,3390,58,N,0,21,57,5,14,16,20,12,30,14,19,5,2,16,23,66,4,23,8,13,11,23,10,11,11,3,16,1
758,2022,151,3376,72,3257,59,N,0,27,57,6,17,12,17,8,24,19,14,11,4,11,27,63,1,8,4,7,11,18,5,15,13,2,17,1
759,2022,153,3376,64,3163,49,N,0,22,60,3,16,17,26,18,23,9,14,6,4,11,22,54,4,16,1,4,3,16,14,14,4,5,21,1


In [5]:
tournament_features = process_detailed_results(NCAATourneyDetailedResults)
tournament_features["Season"] += 1
tournament_features.tail()

,Season,TeamID,ScorePM,ScorePMOpp,FGAPM,FGAPMOpp,FGA3PM,FGA3PMOpp,FTAPM,FTAPMOpp,ORPM,ORPMOpp,DRPM,DRPMOpp,AstPM,AstPMOpp,TOPM,TOPMOpp,StlPM,StlPMOpp,BlkPM,BlkPMOpp,PFPM,PFPMOpp,FGSR,FGSROpp,FG3SR,FG3SROpp,FTSR,FTSROpp
2034,2023,3426,1.775,1.9500,1.5500,1.6250,0.4000,0.5000,0.40,0.6500,0.1750,0.2750,0.6500,0.6750,0.2750,0.1500,0.2250,0.1750,0.1500,0.050,0.1250,0.1500,0.5000,0.3500,0.435484,0.430769,0.437500,0.350000,0.625000,0.576923
2035,2023,3428,1.850,1.8375,1.3375,1.6750,0.6375,0.3625,0.30,0.3125,0.1500,0.2375,0.6375,0.4625,0.4125,0.2625,0.4375,0.2125,0.0750,0.225,0.0375,0.1125,0.3625,0.3000,0.514019,0.470149,0.411765,0.241379,0.708333,0.560000
2036,2023,3437,1.375,1.5125,1.4625,1.3875,0.5875,0.3750,0.30,0.4125,0.1875,0.2875,0.4875,0.7375,0.2750,0.3125,0.2500,0.3750,0.2125,0.175,0.1250,0.0250,0.4500,0.3125,0.350427,0.405405,0.255319,0.266667,0.666667,0.696970
2037,2023,3439,2.025,2.1000,1.5000,1.6000,0.4250,0.9500,0.50,0.2750,0.1750,0.0500,0.7000,0.6250,0.3250,0.4750,0.2500,0.0750,0.0250,0.175,0.0250,0.0250,0.3250,0.4500,0.500000,0.468750,0.352941,0.394737,0.750000,0.818182
2038,2023,3450,1.000,1.2500,1.4000,1.3250,0.5500,0.3500,0.35,0.5500,0.2000,0.1500,0.7750,0.7500,0.1250,0.1750,0.4250,0.2750,0.0500,0.150,0.0500,0.0750,0.5750,0.2750,0.250000,0.264151,0.136364,0.214286,0.642857,0.863636


## Tournement Seeds

In [6]:
NCAATourneySeeds = load_raw_data("NCAATourneySeeds")
NCAATourneySeeds.tail()

,Season,Seed,TeamID,Gender
1535,2022,Z12,3125,1
1536,2022,Z13,3138,1
1537,2022,Z14,3110,1
1538,2022,Z15,3218,1
1539,2022,Z16,3107,1


In [7]:
seed_features = process_seeds(NCAATourneySeeds)
seed_features.tail()

,Season,Seed,TeamID,Gender
1535,2022,12,3125,1
1536,2022,13,3138,1
1537,2022,14,3110,1
1538,2022,15,3218,1
1539,2022,16,3107,1


## Merge features

In [8]:
features = merge_features(season_features, tournament_features, seed_features)
features.tail()

,Season,TeamID,ScorePMReg,ScorePMOppReg,FGAPMReg,FGAPMOppReg,FGA3PMReg,FGA3PMOppReg,FTAPMReg,FTAPMOppReg,ORPMReg,ORPMOppReg,DRPMReg,DRPMOppReg,AstPMReg,AstPMOppReg,TOPMReg,TOPMOppReg,StlPMReg,StlPMOppReg,BlkPMReg,BlkPMOppReg,PFPMReg,PFPMOppReg,FGSRReg,FGSROppReg,FG3SRReg,FG3SROppReg,FTSRReg,FTSROppReg,ScorePMTou,ScorePMOppTou,FGAPMTou,FGAPMOppTou,FGA3PMTou,FGA3PMOppTou,FTAPMTou,FTAPMOppTou,ORPMTou,ORPMOppTou,DRPMTou,DRPMOppTou,AstPMTou,AstPMOppTou,TOPMTou,TOPMOppTou,StlPMTou,StlPMOppTou,BlkPMTou,BlkPMOppTou,PFPMTou,PFPMOppTou,FGSRTou,FGSROppTou,FG3SRTou,FG3SROppTou,FTSRTou,FTSROppTou,Seed,Gender
977,2022,3397,1.734400,1.492800,1.576800,1.596800,0.358400,0.585600,0.492000,0.363200,0.341600,0.228000,0.728000,0.516800,0.363200,0.284000,0.413600,0.318400,0.156000,0.219200,0.148000,0.077600,0.384800,0.464800,0.415525,0.339679,0.303571,0.277322,0.640650,0.676211,1.7750,1.65000,1.56250,1.60000,0.32500,0.66250,0.4750,0.3625,0.4250,0.1875,0.72500,0.4625,0.25000,0.31250,0.45000,0.3000,0.18750,0.2500,0.1750,0.01250,0.38750,0.33750,0.432000,0.359375,0.153846,0.301887,0.789474,0.827586,4,1
978,2022,3400,1.794595,1.400000,1.530502,1.248649,0.334363,0.369884,0.498069,0.457915,0.296525,0.192278,0.522008,0.516602,0.314286,0.253282,0.342857,0.501931,0.247876,0.176834,0.088803,0.080309,0.457915,0.477220,0.433401,0.385281,0.357968,0.281837,0.699225,0.728499,1.5625,1.54375,1.48125,1.45625,0.41875,0.40625,0.4125,0.4125,0.3000,0.2875,0.55625,0.6375,0.24375,0.26875,0.26875,0.3125,0.13125,0.1125,0.0500,0.18125,0.38125,0.44375,0.379747,0.390558,0.328358,0.215385,0.727273,0.772727,2,1
979,2022,3416,1.516964,1.186607,1.392857,1.277679,0.276786,0.483929,0.395536,0.289286,0.292857,0.225000,0.538393,0.466964,0.343750,0.292857,0.373214,0.465179,0.277679,0.188393,0.086607,0.071429,0.341964,0.408036,0.411538,0.333333,0.316129,0.300738,0.715576,0.654321,1.2750,1.55000,1.42500,1.07500,0.32500,0.25000,0.3000,0.5250,0.3250,0.0500,0.55000,0.4750,0.25000,0.47500,0.47500,0.3250,0.12500,0.2000,0.0500,0.17500,0.55000,0.35000,0.368421,0.511628,0.230769,0.300000,0.500000,0.714286,7,1
980,2022,3439,1.755642,1.435019,1.403113,1.476265,0.589105,0.402335,0.396887,0.320623,0.188327,0.215564,0.625681,0.537743,0.355642,0.250584,0.312062,0.319066,0.111284,0.150973,0.087160,0.070039,0.356420,0.438911,0.442041,0.363205,0.363276,0.313346,0.758824,0.737864,1.4750,1.91250,1.40000,1.55000,0.50000,0.30000,0.3125,0.3875,0.1625,0.2750,0.48750,0.6750,0.35000,0.41250,0.35000,0.2875,0.13750,0.1750,0.0375,0.16250,0.45000,0.38750,0.375000,0.491935,0.325000,0.416667,0.840000,0.677419,5,1
981,2022,3450,1.512446,1.502146,1.452361,1.391416,0.536481,0.437768,0.296137,0.346781,0.226609,0.218026,0.539914,0.584549,0.332189,0.278112,0.335622,0.357940,0.196567,0.175966,0.103863,0.079828,0.385408,0.357082,0.388298,0.395435,0.321600,0.329412,0.715942,0.742574,1.3250,1.42500,1.37500,1.45000,0.62500,0.42500,0.3000,0.2500,0.2000,0.2500,0.60000,0.6750,0.27500,0.17500,0.27500,0.2000,0.15000,0.1000,0.1000,0.07500,0.37500,0.27500,0.309091,0.379310,0.320000,0.294118,0.916667,0.800000,8,1


## Build Dataset

In [9]:
from sklearn.model_selection import train_test_split

data = load_raw_data("NCAATourneyCompactResults")
data_train, data_valid = train_test_split(data, random_state=0)

outcomes_train = get_outcomes(data_train)
outcomes_valid = get_outcomes(data_valid)

In [10]:
features_train = merge_outcomes_with_features(outcomes_train, features)
features_valid = merge_outcomes_with_features(outcomes_valid, features)
print(features_train.shape)
features_train.tail()

(794, 117)


,Win,ScorePMRegHigh,ScorePMOppRegHigh,FGAPMRegHigh,FGAPMOppRegHigh,FGA3PMRegHigh,FGA3PMOppRegHigh,FTAPMRegHigh,FTAPMOppRegHigh,ORPMRegHigh,ORPMOppRegHigh,DRPMRegHigh,DRPMOppRegHigh,AstPMRegHigh,AstPMOppRegHigh,TOPMRegHigh,TOPMOppRegHigh,StlPMRegHigh,StlPMOppRegHigh,BlkPMRegHigh,BlkPMOppRegHigh,PFPMRegHigh,PFPMOppRegHigh,FGSRRegHigh,FGSROppRegHigh,FG3SRRegHigh,FG3SROppRegHigh,FTSRRegHigh,FTSROppRegHigh,ScorePMTouHigh,ScorePMOppTouHigh,FGAPMTouHigh,FGAPMOppTouHigh,FGA3PMTouHigh,FGA3PMOppTouHigh,FTAPMTouHigh,FTAPMOppTouHigh,ORPMTouHigh,ORPMOppTouHigh,DRPMTouHigh,DRPMOppTouHigh,AstPMTouHigh,AstPMOppTouHigh,TOPMTouHigh,TOPMOppTouHigh,StlPMTouHigh,StlPMOppTouHigh,BlkPMTouHigh,BlkPMOppTouHigh,PFPMTouHigh,PFPMOppTouHigh,FGSRTouHigh,FGSROppTouHigh,FG3SRTouHigh,FG3SROppTouHigh,FTSRTouHigh,FTSROppTouHigh,SeedHigh,GenderHigh,ScorePMRegLow,ScorePMOppRegLow,FGAPMRegLow,FGAPMOppRegLow,FGA3PMRegLow,FGA3PMOppRegLow,FTAPMRegLow,FTAPMOppRegLow,ORPMRegLow,ORPMOppRegLow,DRPMRegLow,DRPMOppRegLow,AstPMRegLow,AstPMOppRegLow,TOPMRegLow,TOPMOppRegLow,StlPMRegLow,StlPMOppRegLow,BlkPMRegLow,BlkPMOppRegLow,PFPMRegLow,PFPMOppRegLow,FGSRRegLow,FGSROppRegLow,FG3SRRegLow,FG3SROppRegLow,FTSRRegLow,FTSROppRegLow,ScorePMTouLow,ScorePMOppTouLow,FGAPMTouLow,FGAPMOppTouLow,FGA3PMTouLow,FGA3PMOppTouLow,FTAPMTouLow,FTAPMOppTouLow,ORPMTouLow,ORPMOppTouLow,DRPMTouLow,DRPMOppTouLow,AstPMTouLow,AstPMOppTouLow,TOPMTouLow,TOPMOppTouLow,StlPMTouLow,StlPMOppTouLow,BlkPMTouLow,BlkPMOppTouLow,PFPMTouLow,PFPMOppTouLow,FGSRTouLow,FGSROppTouLow,FG3SRTouLow,FG3SROppTouLow,FTSRTouLow,FTSROppTouLow,SeedLow,GenderLow
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2012_1462_1323,True,1.637313,1.514925,1.306716,1.394776,0.480597,0.391791,0.495522,0.344776,0.252985,0.276866,0.585075,0.571642,0.356716,0.305970,0.247015,0.266418,0.127612,0.134328,0.084328,0.072388,0.338806,0.442537,0.431753,0.411985,0.329193,0.310476,0.707831,0.707792,1.575000,1.587500,1.362500,1.400000,0.587500,0.500000,0.562500,0.387500,0.262500,0.212500,0.7250,0.687500,0.350000,0.312500,0.275000,0.250000,0.112500,0.100000,0.125000,0.162500,0.450000,0.487500,0.366972,0.401786,0.297872,0.400000,0.711111,0.677419,7,0,1.749813,1.667416,1.371536,1.417978,0.345318,0.506367,0.559551,0.516105,0.268165,0.274157,0.632210,0.576779,0.331086,0.328090,0.314607,0.314607,0.150562,0.159551,0.092884,0.087640,0.465918,0.478652,0.452758,0.397781,0.349241,0.306213,0.692102,0.744557,1.375000,1.650000,1.275000,1.125000,0.325000,0.300000,0.375000,0.475000,0.275000,0.125000,0.475,0.525000,0.275000,0.325000,0.375000,0.375000,0.075000,0.150000,0.050000,0.050000,0.475000,0.450000,0.411765,0.533333,0.153846,0.416667,0.733333,0.684211,10,0
2015_1455_1242,True,1.773626,1.610989,1.369231,1.448352,0.386813,0.462271,0.589011,0.486447,0.294505,0.304029,0.650549,0.550183,0.330403,0.279121,0.318681,0.291575,0.162637,0.158242,0.125275,0.127473,0.438828,0.483516,0.439807,0.394537,0.375000,0.309033,0.720149,0.668675,1.712500,1.612500,1.387500,1.300000,0.287500,0.500000,0.525000,0.525000,0.337500,0.200000,0.6375,0.500000,0.300000,0.250000,0.350000,0.287500,0.137500,0.225000,0.112500,0.062500,0.450000,0.450000,0.459459,0.413462,0.217391,0.300000,0.714286,0.738095,2,0,1.700800,1.383200,1.351200,1.198400,0.476800,0.356000,0.479200,0.441600,0.288000,0.213600,0.576800,0.540800,0.338400,0.214400,0.232000,0.329600,0.175200,0.098400,0.093600,0.086400,0.412000,0.438400,0.443458,0.400534,0.359060,0.343820,0.691152,0.681159,1.750000,1.437500,1.237500,1.350000,0.500000,0.575000,0.487500,0.500000,0.187500,0.287500,0.675,0.537500,0.387500,0.200000,0.262500,0.212500,0.125000,0.162500,0.112500,0.050000,0.475000,0.400000,0.505051,0.361111,0.400000,0.282609,0.615385,0.600000,7,0
2015_1308_1242,False,1.773626,1.610989,1.369231,1.448352,0.386813,0.462271,0.589011,0.486447,0.294505,0.304029,0.650549,0.550183,0.330403,0.279121,0.318681,0.291575,0.162637,0.158242,0.125275,0.127473,0.438828,0.483516,0.439807,0.

In [11]:
y_train = features_train["Win"]
X_train = features_train.drop("Win", axis=1)
y_valid = features_valid["Win"]
X_valid = features_valid.drop("Win", axis=1)

# Step 4: Train a model


### Setup Hyperparameter Tuning
See https://www.kaggle.com/prashant111/a-guide-on-xgboost-hyperparameters-tuning

In [12]:
from sklearn.metrics import brier_score_loss, roc_auc_score, confusion_matrix
import lightgbm

model = lightgbm.LGBMClassifier(objective="binary", n_estimators=500)
model.fit(X_train, y_train)

preds_train = model.predict(X_train)
preds_valid = model.predict(X_valid)

print("Training Score:", model.score(X_train, y_train))
print("Validation Score:", model.score(X_valid, y_valid))
print("Brier Score Validation:", brier_score_loss(y_valid, preds_valid))
print("ROC AUC Validation:", roc_auc_score(y_valid, preds_valid))
print("Parameters")
print(*(f"- {key}: {value}" for key, value in model.get_params(deep=True).items()), sep="\n")
print("Features")
print(*(f"- {name}: {imp}" for name, imp in sorted(zip(model.feature_name_, model.feature_importances_), key=lambda x: x[1], reverse=True)), sep="\n")


Training Score: 1.0
Validation Score: 0.609375
Brier Score Validation: 0.390625
ROC AUC Validation: 0.609375
Parameters
- boosting_type: gbdt
- class_weight: None
- colsample_bytree: 1.0
- importance_type: split
- learning_rate: 0.1
- max_depth: -1
- min_child_samples: 20
- min_child_weight: 0.001
- min_split_gain: 0.0
- n_estimators: 500
- n_jobs: -1
- num_leaves: 31
- objective: binary
- random_state: None
- reg_alpha: 0.0
- reg_lambda: 0.0
- silent: warn
- subsample: 1.0
- subsample_for_bin: 200000
- subsample_freq: 0
Features
- SeedLow: 333
- SeedHigh: 330
- ScorePMRegHigh: 189
- FTSRRegHigh: 175
- ScorePMRegLow: 167
- FTSRRegLow: 155
- ORPMOppRegLow: 151
- ORPMOppRegHigh: 138
- ScorePMOppRegHigh: 136
- FG3SROppTouHigh: 132
- FGAPMRegHigh: 125
- AstPMOppRegHigh: 124
- ScorePMOppRegLow: 124
- FGA3PMOppRegLow: 121
- FG3SROppTouLow: 121
- StlPMRegHigh: 118
- FGA3PMOppTouHigh: 117
- AstPMOppRegLow: 117
- DRPMOppTouHigh: 114
- StlPMRegLow: 113
- BlkPMOppRegLow: 110
- StlPMOppRegHigh: 10

# Step 5: Submit to the competition

We"ll begin by using the trained model to generate predictions, which we"ll save to a CSV file.

In [13]:
SampleSubmissionWarmup = pd.read_csv("/kaggle/input/warmup-round-march-machine-learning-mania-2023/SampleSubmissionWarmup.csv")

print(SampleSubmissionWarmup.shape)
SampleSubmissionWarmup.tail()

(614319, 2)


,ID,Pred
614314,2022_3469_3471,0.5
614315,2022_3469_3472,0.5
614316,2022_3470_3471,0.5
614317,2022_3470_3472,0.5
614318,2022_3471_3472,0.5


In [14]:
def get_submission_outcomes(sample_submission: pd.DataFrame) -> pd.DataFrame:
    df = sample_submission.copy()
    df.drop("Pred", axis=1, inplace=True)
    df[["Season", "LowID", "HighID"]] = df["ID"].str.split("_", expand=True)
    df[["Season", "LowID", "HighID"]] = df[["Season", "LowID", "HighID"]].astype(int)
    return df

In [15]:
submission_outcomes = get_submission_outcomes(SampleSubmissionWarmup)
print(submission_outcomes.shape)
submission_outcomes.tail()

(614319, 4)


,ID,Season,LowID,HighID
614314,2022_3469_3471,2022,3469,3471
614315,2022_3469_3472,2022,3469,3472
614316,2022_3470_3471,2022,3470,3471
614317,2022_3470_3472,2022,3470,3472
614318,2022_3471_3472,2022,3471,3472


In [16]:
X_submission = merge_outcomes_with_features(submission_outcomes, features, how="left")
print(X_submission.shape)
X_submission.describe()

(614319, 116)


,ScorePMRegHigh,ScorePMOppRegHigh,FGAPMRegHigh,FGAPMOppRegHigh,FGA3PMRegHigh,FGA3PMOppRegHigh,FTAPMRegHigh,FTAPMOppRegHigh,ORPMRegHigh,ORPMOppRegHigh,DRPMRegHigh,DRPMOppRegHigh,AstPMRegHigh,AstPMOppRegHigh,TOPMRegHigh,TOPMOppRegHigh,StlPMRegHigh,StlPMOppRegHigh,BlkPMRegHigh,BlkPMOppRegHigh,PFPMRegHigh,PFPMOppRegHigh,FGSRRegHigh,FGSROppRegHigh,FG3SRRegHigh,FG3SROppRegHigh,FTSRRegHigh,FTSROppRegHigh,ScorePMTouHigh,ScorePMOppTouHigh,FGAPMTouHigh,FGAPMOppTouHigh,FGA3PMTouHigh,FGA3PMOppTouHigh,FTAPMTouHigh,FTAPMOppTouHigh,ORPMTouHigh,ORPMOppTouHigh,DRPMTouHigh,DRPMOppTouHigh,AstPMTouHigh,AstPMOppTouHigh,TOPMTouHigh,TOPMOppTouHigh,StlPMTouHigh,StlPMOppTouHigh,BlkPMTouHigh,BlkPMOppTouHigh,PFPMTouHigh,PFPMOppTouHigh,FGSRTouHigh,FGSROppTouHigh,FG3SRTouHigh,FG3SROppTouHigh,FTSRTouHigh,FTSROppTouHigh,SeedHigh,GenderHigh,ScorePMRegLow,ScorePMOppRegLow,FGAPMRegLow,FGAPMOppRegLow,FGA3PMRegLow,FGA3PMOppRegLow,FTAPMRegLow,FTAPMOppRegLow,ORPMRegLow,ORPMOppRegLow,DRPMRegLow,DRPMOppRegLow,AstPMRegLow,AstPMOppRegLow,TOPMRegLow,TOPMOppRegLow,StlPMRegLow,StlPMOppRegLow,BlkPMRegLow,BlkPMOppRegLow,PFPMRegLow,PFPMOppRegLow,FGSRRegLow,FGSROppRegLow,FG3SRRegLow,FG3SROppRegLow,FTSRRegLow,FTSROppRegLow,ScorePMTouLow,ScorePMOppTouLow,FGAPMTouLow,FGAPMOppTouLow,FGA3PMTouLow,FGA3PMOppTouLow,FTAPMTouLow,FTAPMOppTouLow,ORPMTouLow,ORPMOppTouLow,DRPMTouLow,DRPMOppTouLow,AstPMTouLow,AstPMOppTouLow,TOPMTouLow,TOPMOppTouLow,StlPMTouLow,StlPMOppTouLow,BlkPMTouLow,BlkPMOppTouLow,PFPMTouLow,PFPMOppTouLow,FGSRTouLow,FGSROppTouLow,FG3SRTouLow,FG3SROppTouLow,FTSRTouLow,FTSROppTouLow,SeedLow,GenderLow
count,49144.000000,49144.000000,49144.000000,49144.000000,49144.000000,49144.000000,49144.000000,49144.000000,49144.000000,49144.000000,49144.000000,49144.000000,49144.000000,49144.000000,49144.000000,49144.000000,49144.000000,49144.000000,49144.000000,49144.000000,49144.000000,49144.000000,49144.000000,49144.000000,49144.000000,49144.000000,49144.000000,49144.000000,49144.000000,49144.000000,49144.00000,49144.000000,49144.000000,49144.000000,49144.000000,49144.000000,49144.000000,49144.000000,49144.000000,49144.000000,49144.000000,49144.000000,49144.000000,49144.000000,49144.000000,49144.000000,49144.000000,49144.000000,49144.000000,49144.000000,49144.000000,49144.000000,49144.000000,49144.000000,49144.000000,49144.000000,49144.000000,49144.000000,49499.000000,49499.000000,49499.000000,49499.000000,49499.000000,49499.000000,49499.000000,49499.000000,49499.000000,49499.000000,49499.000000,49499.000000,49499.000000,49499.000000,49499.000000,49499.000000,49499.000000,49499.000000,49499.000000,49499.000000,49499.000000,49499.000000,49499.000000,49499.000000,49499.000000,49499.000000,49499.000000,49499.000000,49499.000000,49499.000000,49499.000000,49499.000000,49499.000000,49499.000000,49499.000000,49499.000000,49499.000000,49499.000000,49499.000000,49499.000000,49499.000000,49499.000000,49499.000000,49499.000000,49499.000000,49499.000000,49499.000000,49499.000000,49499.000000,49499.000000,49499.000000,49499.000000,49499.000000,49499.000000,49499.000000,49499.000000,49499.000000,49499.000000
mean,1.858486,1.590076,1.490356,1.445211,0.507434,0.498032,0.462900,0.400293,0.285958,0.251438,0.646332,0.566554,0.373576,0.296915,0.317045,0.355298,0.179419,0.152058,0.098286,0.075875,0.410129,0.444144,0.451721,0.397679,0.352146,0.318204,0.719876,0.703846,1.702026,1.746000,1.48138,1.463001,0.492698,0.483937,0.414851,0.440886,0.274206,0.263663,0.599813,0.610691,0.321294,0.334457,0.306497,0.299700,0.151111,0.152585,0.092464,0.090276,0.428744,0.413527,0.421098,0.431677,0.327230,0.347176,0.718022,0.726483,6.509849,0.512921,1.875173,1.589861,1.496460,1.451773,0.512861,0.491211,0.459217,0.400281,0.284569,0.253740,0.648357,0.563671,0.388164,0.295589,0.321129,0.360194,0.181053,0.151218,0.099914,0.074257,0.407894,0.441469,0.455768,0.397157,0.351937,0.316320,0.718453,0.702696,1.730575,1.758024,1.484696,1.469402,0.490962,0.468326,0.418485,0.444262,0.269837,0.267539,0.594440,0.606425,0.335586,0.333594,

In [17]:
# Use the model to generate predictions

predictions = model.predict(X_submission)

# Save the predictions to a CSV file
output = pd.DataFrame({"ID": X_submission.index,
                       "Pred": predictions})
output.to_csv("submission.csv", index=False)
print(output.shape)
output.describe()

(614319, 2)


,ID,Pred
count,614319,614319
unique,614319,2
top,2017_1101_1102,True
freq,1,569524
